## Install and Import

In [3]:
pip install sentence_transformers faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pickle
import numpy as np
import pandas as pd
import faiss
import json
import csv
from sentence_transformers import SentenceTransformer
from collections import defaultdict

## Inference on Test set

We used MiniLM embedder to embed the queries in test set; load the document chunk embeddings (of chunk size 512 with MiniLM embedder). We used FAISS to compute the cosine similarity between query and chunk. We applied a weighted algorithm to retrieve the top 10 most relevant documents based on chunks' scores.

In [4]:
# Load embeddings from the merged pickle file
def load_embeddings(embedding_file):
    with open(embedding_file, 'rb') as f:
        document_embeddings = pickle.load(f)
    
    # Convert embeddings from lists to numpy arrays
    for key in document_embeddings:
        document_embeddings[key] = np.array(document_embeddings[key], dtype='float32')
    
    return document_embeddings

# Normalize embeddings for FAISS compatibility
def normalize_embeddings(embeddings):
    norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
    return embeddings / norms

# Build FAISS index for efficient search
def build_faiss_index(document_embeddings):
    embedding_size = len(next(iter(document_embeddings.values())))  # Size of each embedding vector
    index = faiss.IndexFlatIP(embedding_size)  # Inner product index
    
    # Convert embeddings to numpy array and normalize
    doc_ids = list(document_embeddings.keys())
    doc_embeddings_matrix = np.array(list(document_embeddings.values()), dtype='float32')
    doc_embeddings_matrix = normalize_embeddings(doc_embeddings_matrix)  # Normalize embeddings for FAISS

    # Add to FAISS index
    index.add(doc_embeddings_matrix)

    return index, doc_ids, doc_embeddings_matrix

# Embed the query using the SentenceTransformer model
def embed_query(query_text, embedder):
    return embedder.encode(query_text)

# Perform FAISS search and get document-level weighted scores
def search_faiss_weighted(query_embedding, faiss_index, doc_ids, top_n=10):
    query_embedding = np.array([query_embedding], dtype='float32')
    query_embedding = normalize_embeddings(query_embedding)  # Normalize query embedding

    # Search FAISS
    distances, indices = faiss_index.search(query_embedding, top_n * 10)  # Extra results for ranking
    
    # Aggregate scores at the document level
    doc_scores = defaultdict(float)
    for rank, idx in enumerate(indices[0]):
        chunk_id = doc_ids[idx]
        doc_id = chunk_id.split('_chunk_')[0]  # Extract document ID
        weight = 1 / (rank + 1)  # Assign decreasing weight based on rank
        doc_scores[doc_id] += distances[0][rank] * weight  # Weighted score

    # Select top N documents by score
    top_doc_ids = sorted(doc_scores, key=doc_scores.get, reverse=True)[:top_n]
    return top_doc_ids

# Store the query results in a CSV file
def store_candidates(file_name, query_results):
    with open(file_name, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["query_id", "query", "top_doc_ids"])
        for query_id, (query, doc_ids) in enumerate(query_results.items()):
            writer.writerow([query_id, query, json.dumps(doc_ids)])

# Main execution
if __name__ == "__main__":
    # Load embeddings from the merged file
    document_embeddings = load_embeddings("/kaggle/input/512minilm-embedding/merged_chunk_embeddings.pkl")

    # Build the FAISS index
    faiss_index, doc_ids, doc_embeddings_matrix = build_faiss_index(document_embeddings)

    # Initialize SentenceTransformer
    embedder = SentenceTransformer('paraphrase-MiniLM-L6-v2')

    # Load test queries
    test_data_path = "/kaggle/input/dis-project-1-document-retrieval/test.csv"
    test_df = pd.read_csv(test_data_path)
    queries = test_df['query'].tolist()

    # Process each query and perform FAISS search
    query_results = {}
    for query_text in queries:
        query_embedding = embed_query(query_text, embedder)
        top_doc_ids = search_faiss_weighted(query_embedding, faiss_index, doc_ids, top_n=10)
        query_results[query_text] = top_doc_ids

    # Store results in CSV
    store_candidates("/kaggle/working/top_candidates.csv", query_results)

    print("Top 10 candidates stored in 'top_candidates.csv'")


/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Top 10 candidates stored in 'top_candidates.csv'


## Evaluation on Dev set

We applied the same method on the dev split to evaluate the performance (recall@10) as well.

In [ ]:
# Function to compute if the true positive document is in the top N results
def compute_accuracy(dev_data_path, embedder, faiss_index, doc_ids, top_n=200):
    """
    Compute the accuracy by checking if the positive document appears in the top N results.
    """
    dev_df = pd.read_csv(dev_data_path)

    # Counters for accuracy
    correct_count = 0
    total_count = 0

    # Process each query from the dev dataset
    for _, row in dev_df.iterrows():
        query_text = row['query']
        positive_doc = row['positive_docs']  # Assuming this column contains the positive doc ID

        # Embed the query and normalize
        query_embedding = embed_query(query_text, embedder)

        # Retrieve top N results using FAISS
        top_n_doc_ids = search_faiss_weighted(query_embedding, faiss_index, doc_ids, top_n=top_n)

        # Check if the positive_doc is in the top N doc_ids
        if positive_doc in top_n_doc_ids:
            correct_count += 1
        total_count += 1

    # Compute the average accuracy
    accuracy = correct_count / total_count if total_count > 0 else 0
    return accuracy

# Main execution to process dev queries and compute accuracy
if __name__ == "__main__":
    # Load embeddings from the merged file
    document_embeddings = load_embeddings("/kaggle/input/512minilm-embedding/merged_chunk_embeddings.pkl")

    # Build the FAISS index for the document embeddings
    faiss_index, doc_ids, doc_embeddings_matrix = build_faiss_index(document_embeddings)

    # Initialize the SentenceTransformer model (used for embedding queries)
    embedder = SentenceTransformer('paraphrase-MiniLM-L6-v2')

    # Define the number of top results to retrieve
    top_n = 10  # Adjust this as needed

    # Compute accuracy for dev queries
    dev_data_path = "/kaggle/input/dis-project-1-document-retrieval/dev.csv"
    accuracy = compute_accuracy(dev_data_path, embedder, faiss_index, doc_ids, top_n=top_n)

    print(f"Average accuracy for dev queries: {accuracy * 100:.2f}%")
